<a href="https://colab.research.google.com/github/tatiiiiterra/Trabalhos-Tiago-Impacta/blob/master/Aula_06_Estruturas_de_Dados_Complexos_com_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1 - Configurando o Spark no Python**

Etapa responsável por realizar o Download e configuração do Apache Spark.

In [3]:
#Instacao e configuracao do Java 8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Download do Apache Spark
!wget -q http://apache.osuosl.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
#Descompatacao do Apache Spark
!tar xf spark-3.0.1-bin-hadoop3.2.tgz
#Instalacao do Findspark
!pip install -q findspark

**2 - Configurando o Ambiente**

Etapa responsável por configurar o ambiente do Spark

In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"

**3 - Configurando a sessão do Spark**

Etapa responsável por inicializar a sessão do Spark

In [5]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

**4 - Validando a Configuração e Versão do Spark**

Etapa de verificação

In [6]:
import pyspark
print(pyspark.__version__)

3.0.1


**5 - Download dos Dados do Portal da Transparência**

Etapa responsável por realizar o Download das informações de Gastos por meio de cartão de pagamento providas através da API: http://www.portaltransparencia.gov.br/api-de-dados/cartoes?mesExtratoInicio=01%2F2019&mesExtratoFim=12%2F2019&pagina=1





In [55]:
!curl -X GET --header 'Accept: application/json' --header 'chave-api-dados:  32ce7c91f63aa7287c48894ae7860282' 'http://www.portaltransparencia.gov.br/api-de-dados/cartoes?mesExtratoInicio=01%2F2019&mesExtratoFim=12%2F2019&' > consulta_portal_transparencia.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 25988    0 25988    0     0   4778      0 --:--:--  0:00:05 --:--:--  6972


**6 - Carregando os dados no Spark**
Etapa paga a carga de dados no Spark.

In [56]:
#Instancia o contexto do Spark
sc = spark.sparkContext
#Realiza a leitura da nossa consulta realizada na API do Governo Federal sobre Gastos
path = "/content/consulta_portal_transparencia.json"
#Declara dataframe df com o conteudo da api que estava no JSON
df = spark.read.json(path)


**7 - Explorando os dados que foram obtidos**

Etapa de exploração dos dados




In [57]:
#Imprime a estrutura de dados do nosso dataframe
df.printSchema()
#Imprime uma amosta de dados  (para especificar as linhas basta colocar a quantidade entre ())
df.show()

root
 |-- dataTransacao: string (nullable = true)
 |-- estabelecimento: struct (nullable = true)
 |    |-- cnae: struct (nullable = true)
 |    |    |-- classe: string (nullable = true)
 |    |    |-- codigoClasse: string (nullable = true)
 |    |    |-- codigoDivisao: string (nullable = true)
 |    |    |-- codigoGrupo: string (nullable = true)
 |    |    |-- codigoSecao: string (nullable = true)
 |    |    |-- codigoSubclasse: string (nullable = true)
 |    |    |-- divisao: string (nullable = true)
 |    |    |-- grupo: string (nullable = true)
 |    |    |-- secao: string (nullable = true)
 |    |    |-- subclasse: string (nullable = true)
 |    |-- codigoFormatado: string (nullable = true)
 |    |-- complementoEndereco: string (nullable = true)
 |    |-- dataAbertura: string (nullable = true)
 |    |-- descricaoLogradouro: string (nullable = true)
 |    |-- enderecoEletronico: string (nullable = true)
 |    |-- localidadePessoa: string (nullable = true)
 |    |-- municipio: struct

**8 - Tratando e transformando os dados**

In [58]:
dfPrepared = df.selectExpr("dataTransacao","estabelecimento.razaoSocialReceita","portador","unidadeGestora","tipoCartao","tipoCartao","valorTransacao")
dfPrepared.show()

+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+
|dataTransacao|  razaoSocialReceita|            portador|      unidadeGestora|          tipoCartao|          tipoCartao|valorTransacao|
+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+
|   04/11/2019|       NAO SE APLICA|[***.200.821-**, ...|[114606, UNIDADE ...|[1, Cartão de Pag...|[1, Cartão de Pag...|        550,00|
|   18/02/2019|DROGARIA ROSARIO S/A|[***.713.901-**, ...|[443001, AGENCIA ...|[1, Cartão de Pag...|[1, Cartão de Pag...|        179,66|
|   27/07/2019|       NAO SE APLICA|[***.600.679-**, ...|[114629, UNIDADE ...|[1, Cartão de Pag...|[1, Cartão de Pag...|         60,00|
|   13/11/2019|BARREIRAS INFORMA...|[***.628.455-**, ...|[158717, UNIVERSI...|[1, Cartão de Pag...|[1, Cartão de Pag...|        118,75|
|   04/12/2018|MARMONTELLO & MAR...|[***.369.131

**9 - Analise das informações e resultados obtidos**

Etapa onde são realizadas as analises utilizando linguagem SQL para por exemplo descobrir quem mais gastou no ano vigente.

In [60]:
#Declara tabela temporaria no SparkSQL
dfPrepared.createOrReplaceTempView("dfPrepared")

In [61]:
topTravels = spark.sql("select max (valorTransacao) from dfPrepared")
topTravels.show()

+-------------------+
|max(valorTransacao)|
+-------------------+
|              98,46|
+-------------------+



In [62]:
topTravels = spark.sql("select min (valorTransacao) from dfPrepared")
topTravels.show()

+-------------------+
|min(valorTransacao)|
+-------------------+
|           1.000,00|
+-------------------+

